# Example Code for Implement the Noise Model

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

In [ ]:
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.5 MB/s eta 0:00:00


# Implement Noise Model

In [43]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
from qiskit.tools.jupyter import *
import warnings
warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals

In [ ]:
from qiskit.providers.fake_provider import *

In [65]:
import pickle

In [124]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model = pickle.load(file)

In [125]:
noise_model1 = noise.NoiseModel()

In [126]:
noise_modelreal = noise_model1.from_dict(noise_model)

In [127]:
noise_modelreal

<NoiseModel on ['', 'sx', 'cx', 'x', 'reset', 'id', 'measure']>

Transfer the noise model to Pennylane: https://pennylane.ai/blog/2021/05/how-to-simulate-noise-with-pennylane/

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as set of seeds to test results on your side, and the result will be calculated as the average of results from some of those seeds with some hidden seeds. And please use shots as 6000.

In [ ]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
shot = 6000

# Circuit Transpile and Execution

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit_aer.primitives import Estimator
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)
circuit.draw()

┌──────────┐  ┌────┐┌─────────┐     ┌─────────────┐      ┌──────────┐»
q_0: ──┤ Rz(-π/2) ├──┤ √X ├┤ Rz(π/2) ├──■──┤ Ry(0.84756) ├───■──┤ Rz(-π/2) ├»
     ┌─┴──────────┴─┐├───┬┘└─────────┘┌─┴─┐├─────────────┴┐┌─┴─┐└─┬─────┬──┘»
q_1: ┤ Rz(-0.25744) ├┤ S ├────────────┤ X ├┤ Ry(-0.84756) ├┤ X ├──┤ Sdg ├───»
     └──────────────┘└───┘            └───┘└──────────────┘└───┘  └─────┘   »
«         ┌──────┐   ┌─────────┐┌────────────┐
«q_0: ────┤ √Xdg ├───┤ Rz(π/2) ├┤ R(5.735,0) ├
«     ┌───┴──────┴──┐└──┬───┬──┘└────────────┘
«q_1: ┤ Rz(0.25744) ├───┤ I ├─────────────────
«     └─────────────┘   └───┘

Transpile the circuit based on given system model from IBMQ_Montreal.

In [ ]:
system_model = FakeMontreal()

In [ ]:
transpiled_circuit = transpile(circuit, backend=system_model)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp("IIIIIIIIIIIIIIIIIIIIIIIIIXZ")
print(f">>> Observable: {observable.paulis}")

>>> Observable: ['IIIIIIIIIIIIIIIIIIIIIIIIIXZ']


Evaluate on the qiskit Estimator.

In [128]:
estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU',
        'noise_model': noise_modelreal
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    skip_transpilation=True
)

In [129]:
job = estimator.run(transpiled_circuit,observable)
result = job.result()
print(f">>> {result}")

>>> EstimatorResult(values=array([0.44766667]), metadata=[{'shots': 6000, 'variance': 0.7995945555555556, 'simulator_metadata': [{'noise': 'kraus', 'batched_shots_optimization': False, 'measure_sampling': False, 'parallel_shots': 2, 'remapped_qubits': False, 'active_input_qubits': [0, 1], 'num_clbits': 2, 'parallel_state_update': 1, 'num_qubits': 2, 'device': 'CPU', 'input_qubit_map': [[1, 1], [0, 0]], 'method': 'statevector', 'fusion': {'applied': False, 'max_fused_qubits': 2, 'threshold': 14, 'enabled': True}}]}])


## Obtain the Duration of Quantum Circuit

In [ ]:
from qiskit import pulse

Do remember to set the optimization_level to 0 if your circuit is already transpiled!!!

In [ ]:
with pulse.build(system_model) as my_program1:
  with pulse.transpiler_settings(optimization_level=0):
    pulse.call(transpiled_circuit)

In [ ]:
my_program1.duration

4096

# Save the Circuit in QASM Format

In [ ]:
transpiled_circuit.qasm()

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[27];\nrz(-pi/2) q[0];\nsx q[0];\nrz(pi/2) q[0];\nrz(1.3133520832156265) q[1];\ncx q[0],q[1];\nrz(-pi) q[0];\nsx q[0];\nrz(2.2940326955930868) q[0];\nsx q[0];\nsx q[1];\nrz(2.294032695593086) q[1];\nsx q[1];\nrz(-pi) q[1];\ncx q[0],q[1];\nrz(2.1189692017768795) q[0];\nsx q[0];\nrz(-pi/2) q[0];\nrz(-1.313352083215627) q[1];\n'